In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/walmart-recruiting-trip-type-classification/train.csv.zip")
test = pd.read_csv("/kaggle/input/testing/test.csv")

test["VisitNumber"].nunique()

all_data = pd.concat([train,test])
all_data

vis_scan = all_data[["VisitNumber","ScanCount"]]
vis_scan["refund"]  = vis_scan["ScanCount"] == -1
refund_bull = vis_scan.groupby("VisitNumber")["refund"].mean() != 0

all_data['Upc'].value_counts()

all_data["DepartmentDescription"].nunique()

pd.options.display.max_columns = 999

etc = all_data["FinelineNumber"].value_counts()[all_data["FinelineNumber"].value_counts() < 1000].index

all_data["FinelineNumber"] = all_data["FinelineNumber"].apply(lambda x: 0 if x in etc else x)

fineline = pd.get_dummies(all_data["FinelineNumber"])
department = pd.get_dummies(all_data["DepartmentDescription"])

fineline['VisitNumber'] = all_data['VisitNumber']
fineline_sum = fineline.groupby('VisitNumber').sum()



depart_scan_mul=department.mul(all_data['ScanCount'],axis=0) #곱하기 * 대신에 곱할 때는 .mul

depart_scan_mul['VisitNumber'] = all_data['VisitNumber']

depart_scan_mul = depart_scan_mul.groupby("VisitNumber").sum()

all_data = all_data.drop('TripType', 1)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in all_data.columns[all_data.dtypes == object]:
    all_data[i] = le.fit_transform(list(all_data[i]))

count = all_data.groupby("VisitNumber")["Upc"].agg(['count'])
weekday= all_data.groupby("VisitNumber")["Weekday"].mean()
scancount = all_data.groupby("VisitNumber")["ScanCount"].agg(["sum","mean","std","max","min","median"])

count.columns = ['Upc_count']

all_data.isnull().sum()

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure()
sns.countplot(all_data["Weekday"])

VisitNum = pd.DataFrame(all_data["VisitNumber"].unique() , columns=["VisitNumber"])
VisitNum = pd.merge(VisitNum, count, on="VisitNumber",how="left")
VisitNum = pd.merge(VisitNum, weekday, on="VisitNumber",how="left")
VisitNum = pd.merge(VisitNum, fineline_sum, on="VisitNumber",how="left")
VisitNum = pd.merge(VisitNum, depart_scan_mul, on="VisitNumber",how="left")
VisitNum = pd.merge(VisitNum, scancount, on="VisitNumber",how="left")
VisitNum = pd.merge(VisitNum, refund_bull, on="VisitNumber",how="left")

triptype = train.groupby('VisitNumber')["TripType"].mean()

train2 = VisitNum[:len(triptype)]
test2 = VisitNum[len(triptype):]

from catboost import CatBoostClassifier 
cat = CatBoostClassifier(iterations= 4000,task_type="GPU")
cat.fit(train2,triptype)
result = cat.predict_proba(test2)

sub = pd.read_csv("/kaggle/input/walmart-recruiting-trip-type-classification/sample_submission.csv.zip")
sub.iloc[:,1:] = result
sub.to_csv("triptype.csv", index=0)